In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier as RFC 
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import *
import random

In [2]:
df = pd.read_csv('Dataset/rgb_img_dataset_test.csv')

In [3]:
df_test = df[(df['filename']=='thai-street-fashion-output/test/test00066.png')|
             (df['filename']=='harajuku-kawaii-street-fashion-output/test/test00011.png')]

In [4]:
df = df.drop('filename', axis=1)

In [5]:
df

,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,...,vest,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress,label
0,16-31,16-31,16-31,0.303225,0-15,0-15,0-15,0.235109,16-31,0-15,...,0.987999,0.0,0.0,0.000000,0.887868,0.0,0.0,0.0,0.0,0
1,16-31,16-31,16-31,0.176878,240-255,240-255,240-255,0.091829,224-239,224-239,...,0.000000,0.0,0.0,0.985530,0.930180,0.0,0.0,0.0,0.0,0
2,224-239,224-239,224-239,0.092203,208-223,208-223,208-223,0.082258,240-255,224-239,...,0.998924,0.0,0.0,0.894071,0.000000,0.0,0.0,0.0,0.0,0
3,32-47,32-47,48-63,0.063760,0-15,32-47,112-127,0.059953,0-15,48-63,...,0.996589,0.0,0.0,0.000000,0.939734,0.0,0.0,0.0,0.0,0
4,0-15,0-15,0-15,0.127613,160-175,176-191,192-207,0.112392,144-159,176-191,...,0.970125,0.0,0.0,0.732579,0.000000,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,16-31,0-15,0-15,0.382456,16-31,16-31,0-15,0.152202,96-111,96-111,...,0.952845,0.0,0.0,0.000000,0.993711,0.0,0.0,0.0,0.0,0
603,32-47,32-47,32-47,0.196324,192-207,192-207,192-207,0.066635,176-191,176-191,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0
604,128-143,112-127,64-79,0.030152,144-159,112-127,48-63,0.028196,144-159,112-127,...,0.786874,0.0,0.0,0.877317,0.000000,0.0,0.0,0.0,0.0,0
605,0-15,32-47,32-47,0.134387,16-31,32-47,32-47,0.129191,16-31,32-47,...,0.000000,0.0,0.0,0.999213,0.000000,0.0,0.0,0.0,0.0,0


In [6]:
def convert_range_to_order(val):
    result = int(val.split('-')[0])/16
    return int(result)

In [7]:
for col_name in df.columns:
    if('_range_' in col_name):
        df[col_name] = df[col_name].apply(lambda x: convert_range_to_order(x))
        df_test[col_name] = df_test[col_name].apply(lambda x: convert_range_to_order(x))

In [8]:
df

,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,...,vest,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress,label
0,1,1,1,0.303225,0,0,0,0.235109,1,0,...,0.987999,0.0,0.0,0.000000,0.887868,0.0,0.0,0.0,0.0,0
1,1,1,1,0.176878,15,15,15,0.091829,14,14,...,0.000000,0.0,0.0,0.985530,0.930180,0.0,0.0,0.0,0.0,0
2,14,14,14,0.092203,13,13,13,0.082258,15,14,...,0.998924,0.0,0.0,0.894071,0.000000,0.0,0.0,0.0,0.0,0
3,2,2,3,0.063760,0,2,7,0.059953,0,3,...,0.996589,0.0,0.0,0.000000,0.939734,0.0,0.0,0.0,0.0,0
4,0,0,0,0.127613,10,11,12,0.112392,9,11,...,0.970125,0.0,0.0,0.732579,0.000000,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,1,0,0,0.382456,1,1,0,0.152202,6,6,...,0.952845,0.0,0.0,0.000000,0.993711,0.0,0.0,0.0,0.0,0
603,2,2,2,0.196324,12,12,12,0.066635,11,11,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0
604,8,7,4,0.030152,9,7,3,0.028196,9,7,...,0.786874,0.0,0.0,0.877317,0.000000,0.0,0.0,0.0,0.0,0
605,0,2,2,0.134387,1,2,2,0.129191,1,2,...,0.000000,0.0,0.0,0.999213,0.000000,0.0,0.0,0.0,0.0,0


In [9]:
df['label'].value_counts()

0    455
1    152
Name: label, dtype: int64

In [10]:
def model_training(df, training_size, param): 
    # training_size is ratio of training data for train-test split (float)
    # param is set of parameter for model training (dict)
    
    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = tts(X, y, train_size=training_size, random_state=42)
    
    sm = SMOTE(sampling_strategy = 1, random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train.ravel())
    
    clf = RFC(max_depth=param['max_depth'], 
              criterion=param['criterion'],
              min_impurity_decrease=param['min_impurity_decrease'], random_state=42).fit(X_res, y_res)
    # clf = GBC(set_of_parameter)
    y_pred = clf.predict(X_test)
    
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))
    print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred)))
    print('Precision: {:.4f}'.format(precision_score(y_test, y_pred)))
    print('Recall: {:.4f}'.format(recall_score(y_test, y_pred)))
    
    return clf

In [11]:
def random_parameter(param_name):
    
    if(param_name=='max_depth'):
        return random.randint(1, 10)
    
    if(param_name=='criterion'):
        tmp = random.randint(1, 2)
        if(tmp==1):
            return 'gini'
        elif(tmp==2):
            return 'entropy'
            
    if(param_name=='min_impurity_decrease'):
        return random.random()/10

In [12]:
param = {'max_depth': random_parameter('max_depth'),
         'criterion': random_parameter('criterion'),
         'min_impurity_decrease': random_parameter('min_impurity_decrease')}

In [13]:
model = model_training(df, 0.75, param)

Confusion Matrix
[[97 20]
 [ 8 27]]
Accuracy: 0.8158
Precision: 0.5745
Recall: 0.7714


In [190]:
models = []
for i in range(5000):
    param = {'max_depth': random_parameter('max_depth'),
             'criterion': random_parameter('criterion'),
             'min_impurity_decrease': random_parameter('min_impurity_decrease')}
    model = model_training(df, 0.8, param)
    
    output = model.predict(x_input)
    models.append(model)
    if(np.array_equiv(output, [1, 1])):
        break
    

Confusion Matrix
[[58 34]
 [ 4 26]]
Accuracy: 0.6885
Precision: 0.4333
Recall: 0.8667
Confusion Matrix
[[39 53]
 [ 3 27]]
Accuracy: 0.5410
Precision: 0.3375
Recall: 0.9000


In [191]:
df_test = df_test.reset_index().drop('index', axis=1)
filename = df_test['filename']
x_input = df_test.drop(['filename', 'label'], axis=1)
output = models[-1].predict(x_input)
# output = np.where(output>=0.5, 1, 0)

In [192]:
filename

0    harajuku-kawaii-street-fashion-output/test/tes...
1        thai-street-fashion-output/test/test00066.png
Name: filename, dtype: object

In [193]:
x_input

,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,...,long_sleeved_outwear,vest,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress
0,4,0,15,0.047448,3,1,0,0.024838,2,1,...,0.0,0.0,0.0,0.0,0.965911,0.981742,0.0,0.0,0.0,0.0
1,15,14,13,0.151879,15,15,14,0.055616,15,13,...,0.0,0.0,0.0,0.0,0.780346,0.000000,0.0,0.0,0.0,0.0


In [194]:
output

array([1, 1], dtype=int64)

In [195]:
output_tab = pd.concat([filename, x_input], axis=1)
output_tab['predicted_class'] = output

In [197]:
output_tab.to_csv('example_predict.csv', index=0)